<a href="https://colab.research.google.com/github/kimsunlim/TIL/blob/master/chapter4_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 다양한 분류 알고리즘 

### 1. 로지스틱 회귀

Q: 럭키백에 들어가는 생선은 7개 특성치는 길이,높이,두께,대각선길이, 무게 총5개로 7개 생선에 대한 뽑힐수 있는 확률 출력

1) 최근접 이웃 
:타깃 주변 이웃을 찾아서 클래스 비율을 확률로 출력 하면 되지 않을까?

In [ ]:
import pandas as pd
fish=pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [ ]:
# unique 고유의값 추출
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


타깃: speices >> 확률구하고자 하는 열 




나머지 5개 열은 입력데이터로 지정

In [ ]:
fish_input=fish[['Weight',	'Length',	'Diagonal',	'Height',	'Width']].to_numpy() #넘파이 배열로 꼭 변환!

In [ ]:
print(fish_input[:5])  #잘 선택 되었는지 확인 

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [ ]:
fish_target=fish['Species'].to_numpy()

타깃 데이터 = fish_target

입력데이터 = fish_input
으로 데이터 지정 

In [ ]:
## 데이터셋 분리 
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target =train_test_split(fish_input, fish_target, random_state=42)

In [ ]:
##스케일링 >>입력데이터 정규화 (5개의 각자 다른 데이터 정규화하는 것)
from sklearn.preprocessing import StandardScaler
ss= StandardScaler() #변수 설정 
ss.fit (train_input) #훈련적용 

train_scaled= ss.transform(train_input) #표준화 스케일링 변환 
test_scaled= ss.transform(test_input)

In [ ]:
#k-최근접 이웃 분류기 확률 예측 
from sklearn.neighbors import KNeighborsClassifier 
kn=KNeighborsClassifier(n_neighbors=3) #최근접 이웃개수 k=3으로 설정 
kn.fit(train_scaled,train_target) #훈련데이터 fit 
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [ ]:
## 정렬된 타깃값 확인 
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [ ]:
## 분류모델은 predict_proba()로 클래스 확률값을 반환함 
import numpy as np 
proba =kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

#첫번쨰 열이 Bream에 대한 확률, 두번쨰 열이 parkki에 대한 확률

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


- 그러나 이웃의 수를 3개만 지정해서 1/3,2/3,3/3 이 전부임 
- 확률이라고 말하기 좀 그럼 

 2) **로지스틱 회귀**
 
 
 : 이름은 회귀이지만 분류모델이다 
 
 
 +시그모이드 함수 


- 로지스틱회귀로 이진분류

In [ ]:

## 이진분류 수행  >> 0.5보다 크면 양성, 작으면 음성 
##도미와 빙어 행만 True값으로 반환 

bream_smelt_indexes = (train_target=='Bream')| (train_target=='Smelt')
train_bream_smelt= train_scaled[bream_smelt_indexes]
target_bream_smelt= train_target[bream_smelt_indexes]


In [ ]:
#로지스틱 
from sklearn.linear_model import LogisticRegression 
lr=LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [ ]:
#처음 5개 샘플 예측 
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [ ]:
print(lr.predict_proba(train_bream_smelt[:5]))
#위의 예측의 확률 값 
#첫번쨰 열이 음성 클래스에 대한 확률, 두번째 열은 양성 클래스에 대한 확률
#음성 도미 ,양성 빙어

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


- 로지스틱회귀로 다중분류


: 로지스틱으로 7개의 생선 분류

In [ ]:
lr= LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))
#정확도 양호 

0.9327731092436975
0.925


In [ ]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [ ]:
proba =lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


### 2. 확률적 경사 하강법 
- 경사하강법을 이해하고 대량 데이터에서 분류모델 훈련하는 법을 공부해보자


In [ ]:
import pandas as pd 
fish=pd.read_csv('http://bit.ly/fish_csv_data')

In [ ]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [ ]:
fish_input=fish[['Weight','Length',	'Diagonal',	'Height',	'Width']].to_numpy()
fish_target=fish['Species'].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [ ]:
from sklearn.model_selection import train_test_split 
train_input, test_input,train_target, test_target= train_test_split(fish_input, fish_target, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train_input)
train_scaled= ss.transform(train_input)
test_scaled= ss.transform(test_input)

In [ ]:
## 확률적 경사하강법 
from sklearn.linear_model import SGDClassifier


In [ ]:
sc= SGDClassifier(loss='log',max_iter=10, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))


0.773109243697479
0.775


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


In [ ]:
sc.partial_fit(train_scaled, train_target) #1 에포크씩 이서서 훈련
print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))

0.8151260504201681
0.85


In [ ]:
sc= SGDClassifier(loss='log',max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))


0.957983193277311
0.925
